# Download last.fm listening history

  - To see this analysis live, check out my article ["Analyzing Last.fm Listening History"](http://geoffboeing.com/2016/05/analyzing-lastfm-history/)
  - My last.fm page: http://www.last.fm/user/gboeing
  - API documentation: http://www.last.fm/api
  - For anything more complicated, you might use this Python wrapper for the API: https://github.com/pylast/pylast
  
This tool separately downloads your all-time most played tracks, artists, and albums. Then it downloads all of your scrobbles in order of recency. Each of these 4 data sets are saved to separate CSV files. It downloads the "all-time most" data separately because (at least for my data) my massive iTunes history scrobble-upload in 2007 is included in the all-time most played tracks/artists/albums but is excluded from the recent tracks API endpoint. For accurate analysis of my all-time scrobbles, I need to look at those separate all-time lists, or else 4 years of listening history (from iTunes) is ignored in the calculations.

Replace the "from keys import..." line of code with two new lines of code (replace placeholder values with your actual values):
  1. key=YOUR-LASTFM-API-KEY
  2. username=YOUR-LASTFM-USERNAME

In [1]:
import requests, json, time, pandas as pd
from keys import lastfm_api_key as key, lastfm_user_name as username

C:\Users\Danny\Anaconda\lib\site-packages\pandas\computation\__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [2]:
# how long to pause between consecutive API requests
pause_duration = 0.2

## First get your all-time most played tracks, artists, and albums

In [3]:
url = 'https://ws.audioscrobbler.com/2.0/?method=user.get{}&user={}&api_key={}&limit={}&extended={}&page={}&format=json'
limit = 200 #api lets you retrieve up to 200 records per call
extended = 0 #api lets you retrieve extended data for each track, 0=no, 1=yes
page = 1 #page of results to start retrieving at

In [4]:
method = 'toptracks'
request_url = url.format(method, username, key, limit, extended, page)
artist_names = []
track_names = []
play_counts = []
response = requests.get(request_url).json()
for item in response[method]['track']:
    artist_names.append(item['artist']['name'])
    track_names.append(item['name'])
    play_counts.append(item['playcount'])

top_tracks = pd.DataFrame()
top_tracks['artist'] = artist_names
top_tracks['track'] = track_names
top_tracks['play_count'] = play_counts
top_tracks.to_csv('data/lastfm_top_tracks.csv', index=None, encoding='utf-8')
top_tracks.head()

,artist,track,play_count
0,Linkin Park,Iridescent,64
1,Tim Minchin,The Fence,54
2,Mumford & Sons,I Gave You All,45
3,Placebo,Bright Lights,42
4,The Police,Message in a Bottle,42


In [5]:
method = 'topartists'
request_url = url.format(method, username, key, limit, extended, page)
artist_names = []
play_counts = []
response = requests.get(request_url).json()
for item in response[method]['artist']:
    artist_names.append(item['name'])
    play_counts.append(item['playcount'])

top_artists = pd.DataFrame()
top_artists['artist'] = artist_names
top_artists['play_count'] = play_counts
top_artists.to_csv('data/lastfm_top_artists.csv', index=None, encoding='utf-8')
top_artists.head()

,artist,play_count
0,The Offspring,783
1,Linkin Park,731
2,Placebo,682
3,David Mitchell,651
4,Mumford & Sons,611


In [6]:
method = 'topalbums'
request_url = url.format(method, username, key, limit, extended, page)
artist_names = []
album_names = []
play_counts = []
response = requests.get(request_url).json()
for item in response[method]['album']:
    artist_names.append(item['artist']['name'])
    album_names.append(item['name'])
    play_counts.append(item['playcount'])

top_albums = pd.DataFrame()
top_albums['artist'] = artist_names
top_albums['album'] = album_names
top_albums['play_count'] = play_counts
top_albums.to_csv('data/lastfm_top_albums.csv', index=None, encoding='utf-8')
top_albums.head()

,artist,album,play_count
0,Mumford & Sons,Babel,306
1,Imagine Dragons,Smoke + Mirrors (Limited Super Deluxe Edition),263
2,30 Seconds to Mars,This Is War,260
3,Tim Minchin,Live at the O2,238
4,Placebo,Battle for the Sun,226


## Now get all your scrobbles

Last.fm provides this 'recenttracks' API method to get 'all' scrobbles. However, it seems to be pretty spotty for data from circa 2007. The best way to determine top tracks, artists, albums is with the cells above. However, the code below retrieves time series data of all scrobbles (but with the caveat of spotty data from 2007 and earlier).

Sample URL: https://ws.audioscrobbler.com/2.0/?method=user.getrecenttracks&user=gboeing&api_key={}&limit=1&extended=0&page=1&format=json

In [7]:
def get_scrobbles(method='recenttracks', username=username, key=key, limit=200, extended=0, page=1, pages=0):
    '''
    method: api method
    username/key: api credentials
    limit: api lets you retrieve up to 200 records per call
    extended: api lets you retrieve extended data for each track, 0=no, 1=yes
    page: page of results to start retrieving at
    pages: how many pages of results to retrieve. if 0, get as many as api can return.
    '''
    # initialize url and lists to contain response fields
    url = 'https://ws.audioscrobbler.com/2.0/?method=user.get{}&user={}&api_key={}&limit={}&extended={}&page={}&format=json'
    responses = []
    artist_names = []
    artist_mbids = []
    album_names = []
    album_mbids = []
    track_names = []
    track_mbids = []
    timestamps = []
    
    # make first request, just to get the total number of pages
    request_url = url.format(method, username, key, limit, extended, page)
    response = requests.get(request_url).json()
    total_pages = response[method]['@attr']['totalPages']
    if pages > 0:
        total_pages = min([total_pages, pages])
        
    print '{} total pages to retrieve'.format(total_pages)
    
    # request each page of data one at a time
    for page in range(1, int(total_pages) + 1, 1):
        if page % 10 == 0: print page,
        time.sleep(pause_duration)
        request_url = url.format(method, username, key, limit, extended, page)
        responses.append(requests.get(request_url))
    
    # parse the fields out of each scrobble in each page (aka response) of scrobbles
    for response in responses:
        scrobbles = response.json()
        for scrobble in scrobbles[method]['track']:
            # only retain completed scrobbles (aka, with timestamp and not 'now playing')
            if 'date' in scrobble.keys():
                artist_names.append(scrobble['artist']['#text'])
                artist_mbids.append(scrobble['artist']['mbid'])
                album_names.append(scrobble['album']['#text'])
                album_mbids.append(scrobble['album']['mbid'])
                track_names.append(scrobble['name'])
                track_mbids.append(scrobble['mbid'])
                timestamps.append(scrobble['date']['uts'])
                
    # create and populate a dataframe to contain the data
    df = pd.DataFrame()
    df['artist'] = artist_names
    df['artist_mbid'] = artist_mbids
    df['album'] = album_names
    df['album_mbid'] = album_mbids
    df['track'] = track_names
    df['track_mbid'] = track_mbids
    df['timestamp'] = timestamps
    df['datetime'] = pd.to_datetime(df['timestamp'].astype(int), unit='s')
    
    return df

In [8]:
# get all scrobbled tracks ever, in order of recency
scrobbles = get_scrobbles()

89 total pages to retrieve
10 20 30 40 50 60 70 80


In [9]:
# save the dataset
scrobbles.to_csv('data/lastfm_scrobbles.csv', index=None, encoding='utf-8')
print '{:,} total rows'.format(len(scrobbles))
scrobbles.head()

17,763 total rows


,artist,artist_mbid,album,album_mbid,track,track_mbid,timestamp,datetime
0,Led Zeppelin,d6e0e274-8e19-44ce-b5b2-52c12f8a674a,Greatest Hits Disc 1,,Four Sticks,c5da2d84-c7d2-4bee-9416-a94394b11af1,1490048324,2017-03-20 22:18:44
1,The Temper Trap,bb6eaced-69dd-4636-829f-9f987acc4fc3,Conditions,2342ef67-07f8-4fc7-b7c5-a15d20bccac1,Sweet Disposition,e1a41054-6110-47b8-9e29-ee39fcfd7cdc,1490048091,2017-03-20 22:14:51
2,The John Butler Trio,7952b266-9fd4-4a09-a324-7dc84f11b5fc,Grand National,ce869c63-e3a8-4c58-8c20-b279417cd9c4,Caroline,d85d58d2-b0a8-44b0-a8c1-39d792755771,1490047931,2017-03-20 22:12:11
3,The Police,9e0e2b01-41db-4008-bd8b-988977d6019a,The Police,48160058-d239-4b0b-9969-47f73e6cf86f,Every Breath You Take,4bd93e75-bc94-46b8-854f-8c7cea071823,1490047676,2017-03-20 22:07:56
4,The Royal Philharmonic Orchestra,299597ae-3a20-4bf1-973e-03704af69f6b,Symphonic Rock,,Imagine - John Lennon,,1490047424,2017-03-20 22:03:44
